In [5]:
import numpy as np
import glob
from src import parseTrackletXML as pt_XML

In [6]:
lidar_points = glob.glob('./velodyne_points/data/*.bin')


print(len(lidar_points))

154


In [36]:
def load_tracklet(xml_path, n_frame):
    tracklets = pt_XML.parseXML(xml_path)
    objects = []  #grouped by frames
    
    twoPi = 2.*np.pi
    
    f_tracklet = {}
    f_type = {}
    #for i in range(n_frame):
    #    frame_tracklets[i] = []
    #    frame_tracklets_types[i] = []
    
    
    # refered to parseTrackletXML.py example fuction
    # loop over tracklets
    for tracklet in tracklets:
        
        # this part is inspired by kitti object development kit matlab code: computeBox3D
        h, w, l = tracklet.size
        trackletBox = np.array([ # in velodyne coordinates around zero point and without orientation yet\
        [-l/2, -l/2,  l/2, l/2, -l/2, -l/2,  l/2, l/2], \
        [ w/2, -w/2, -w/2, w/2,  w/2, -w/2, -w/2, w/2], \
        [ 0.0,  0.0,  0.0, 0.0,    h,     h,   h,   h]])
        
        # loop over all data in tracklet
        for translation, rotation, state, occlusion, truncation, amtOcclusion, amtBorders, absoluteFrameNumber in tracklet:

            # determine if object is in the image; otherwise continue
            if truncation not in (pt_XML.TRUNC_IN_IMAGE, pt_XML.TRUNC_TRUNCATED):
                continue

            # re-create 3D bounding box in velodyne coordinate system
            yaw = rotation[2]   # other rotations are 0 in all xml files I checked
            assert np.abs(rotation[:2]).sum() == 0, 'object rotations other than yaw given!'
            rotMat = np.array([\
              [np.cos(yaw), -np.sin(yaw), 0.0], \
              [np.sin(yaw),  np.cos(yaw), 0.0], \
              [        0.0,          0.0, 1.0]])

            cornerPosInVelo = np.dot(rotMat, trackletBox) + np.tile(translation, (8,1)).T        
            
            if absoluteFrameNumber in f_tracklet:
                f_tracklet[absoluteFrameNumber] += [cornerPosInVelo]
                f_type[absoluteFrameNumber] += [tracklet.objectType]
            else:
                f_tracklet[absoluteFrameNumber] = [cornerPosInVelo]
                f_type[absoluteFrameNumber] = [tracklet.objectType]
            
    return f_tracklet, f_type
        
    
xml_path = "./tracklet_labels.xml"    
tracklet, type = load_tracklet(xml_path, len(lidar_points))

parsing tracklet file ./tracklet_labels.xml
file contains 15 tracklets
loaded 15 tracklets


In [16]:
def projectToImage(points_3d, K):
    points_2d = np.dot(K, points_3d[:3,:])
    points_2d[0,:]=points_2d[0,:]/points_2d[2,:]
    points_2d[1,:]=points_2d[1,:]/points_2d[2,:]
    return points_2d[:2,:]
